# ECS529U Algorithms and Data Structures                  
# Lab sheet 10

This lab gets you to work with dynamic programming algorithms and also practically 
compare their efficiency by testing them on randomly generated inputs. 

**Marks (max 5):**  Questions 1-5: 1 each

## Question 1

We define the Thribonacci sequence of numbers by the following function _thrib_:

- _thrib_(0) = 2
- _thrib_(1) = 1
- _thrib_(2) = 0
- _thrib_(n) = 3 _thrib_(n-1) + 2 _thrib_(n-2) + _thrib_(n-3) ,  if n > 2

Write a recursive Python function 

    def thrib(n)
    
that, on input `n`, returns _thrib_(`n`). 

Then, change your function into a dynamic programming one: 

    def thribDP(n)

using memoisation.

In [4]:
def thrib(n):
    if n > 2:
        return 3*thrib(n-1) + 2*thrib(n-2) + thrib(n-3)
    elif n == 0:
        return 2
    elif n == 1:
        return 1
    elif n == 2:
        return 0

def thribDP(n):
    memo = {}
    return thribCalc(n, memo)

def thribCalc(n, memo):
    if n in memo:
        return memo[n]

    if n == 0:
        memo[n] = 2
    elif n == 1:
        memo[n] = 1
    elif n == 2:
        memo[n] = 0
    else:
        memo[n] = 3*thribCalc(n-1, memo) + 2*thribCalc(n-2, memo) + thribCalc(n-3, memo)
    return memo[n]
print(thrib(3))
print(thribDP(3))

4
4


## Question 2

Change your DP function from Question 1 into a dynamic programming bottom-up one:

    def thribDPBU(n)

using iteration

In [15]:
def thribDPBU(n):
    if n == 0:
        return 2
    elif n == 1:
        return 1
    elif n == 2:
        return 0
    
    A = [None] * (n+1)

    A[0], A[1], A[2] = 2,1,0

    for i in range(3, n+1):
        A[i] = 3*A[i-1] + 2*A[i-2] + A[i-3]
    
    return A[n]

print(thribDPBU(15))
        

18583637


# Question 3

Write Python functions:

    def coinSplitTime(n)
    def coinSplitDPTime(n)
    def coinSplitDPBUTime(n)
    
that run `coinSplit(n)`, `coinSplitDP(n)` and `coinSplitDPBU(n)` respectively on input `n` 
and return the time taken for each of them to return. 

Test your timing functions on values 10, 100, 1000, 10000 for `n` and fill in the next table. 

Use these two choices for `coin`:

1. `coin1 = [200, 100, 50, 20, 5, 2, 1]`            
2. `coin2 = [200, 199, 198, ..., 3, 2, 1]`

To avoid waiting forever, if a run takes more than e.g. 15 seconds then kill it and fill in "timeout" in the table. **Note you need to fill in the table to get marks in this question!**

| value n/ coin array  |  10/ coin1 | 100/ coin1 | 1000/ coin1 | 10000/ coin1 | 10/ coin2 | 100/ coin2 | 1000/ coin2 | 10000/ coin2 |
|:------------|------|-----|------|-------|--------|--------|--------|--------|
| coinSplit time (sec)| -1.0013580322265625e-05 |-0.0014379024505615234| timeout | max recursion |-0.0001163482666015625| timeout | timeout | max recursion|               
| coinSplitDP time (sec)| -2.002716064453125e-05 | -0.00011801719665527344 | -0.0015881061553955078 | max recursion |-9.584426879882812e-05|-0.003467559814453125|-0.10828495025634766|max recursion|                
| coinSplitDPBU time (sec)| -3.1948089599609375e-05 | -0.00022292137145996094 | -0.0022079944610595703 | -0.021481037139892578 |-0.00028395652770996094|-0.002610921859741211|-0.05002903938293457|-0.5149381160736084|                

In [25]:
import time

# add a counter so that we can check how many recursive calls are made
c1 = 0

def coinSplitTime(m): # inefficient solution
    t = time.time()
    result = coinSplitRec(m,0)
    return t - time.time()
    

def coinSplitRec(m, lo):  # split m using the coin[lo:]
    global c1
    c1 += 1
    if m == 0:
        return 0
    if lo == len(coin)-1:
        return m
    withoutIt = coinSplitRec(m,lo+1)
    if coin[lo] <= m:
        withIt = 1 + coinSplitRec(m-coin[lo],lo)
        if withIt < withoutIt:
            return withIt
    return withoutIt



c2 = 0

def coinSplitDPTime(m):
    # memo is a hash map with keys of the form (m,lo)
    t = time.time()
    memo = {}
    result = coinSplitMem(m,0,memo)
    return t - time.time()

def coinSplitMem(m, lo, memo):  # split m using the coin[lo:]
    global c2
    c2 += 1    
    if (m,lo) in memo:  # have I already calculated the best split for (m,lo)?
        return memo[m,lo]
    if m == 0:
        memo[m,lo] = 0
    elif lo == len(coin)-1:
        memo[m,lo] = m
    else:
        withoutIt = coinSplitMem(m,lo+1,memo)
        memo[m,lo] = withoutIt
        if coin[lo] <= m:
            withIt = 1 + coinSplitMem(m-coin[lo],lo,memo)
            if withIt < withoutIt:
                memo[m,lo] = withIt
    return memo[m,lo]


def coinSplitDPBUTime(n):
    t = time.time()
    result = coinSPlitDPBUCalc(n)
    return t - time.time()

def coinSPlitDPBUCalc(n):
    dp = [[999999999 for _ in range(n + 1)] for _ in range(len(coin) + 1)] # storing the subset of coins and the remaining amount
    for i in range(len(coin) + 1):
        dp[i][0] = 0

    for i in range(1, len(coin) + 1): # filling the 2D array bottom up.
        for j in range(1, n + 1):
            dp[i][j] = dp[i-1][j]
            if coin[i-1] <= j: # Can I use the i-th coin?
                dp[i][j] = min(dp[i][j], 1 + dp[i][j - coin[i-1]])
    
    return dp[len(coin)][n]

#coin = [200, 100, 50, 20, 10, 5, 2, 1]
coin = [i for i in range(200, 0, -1)]
testcase = [10,100,1000,10000]
for i in testcase:
    print(coinSplitTime(i))
    

-0.0001163482666015625


KeyboardInterrupt: 

# Question 4

Similarly to Question 4 of Lecture 10, suppose we have a bag and we want to fill it with books.
The bag can take at most `w` kilos of weight, while
the weights of our books are given by an array
`bkWeight` (e.g. `bkWeight[0]` is the weight of the
first book, etc.). Each book has a value, given by an
array `bkVal` (e.g. `bkVal[0]` is the value of the first
book, etc.). 

Write a dynamic programming function

    def maxBooks(w, bkWeight, bkVal)

which returns a pair `(v,A)` where `v` is the maximum value of books that we can fill our bag with and `A` is an array containing the corresponding books in order (i.e. their indices). For example, the following code:

    bkWeight = [1,1,3,4,6,12,33,45,50]
    bkVal = [1,2,5,1,10,20,24,5,60]
    print(maxBooksVal(100,bkWeight,bkVal))
    
should return `(112, [0, 1, 2, 5, 6, 8])`. Assume `bkWeight` is sorted. *Hint:* work as suggested in Question 6, Lecture 10.

In [51]:
def maxBooks(w, bkWeight, bkVal):
    n = len(bkVal)
    dp = [[0 for _ in range(w + 1)] for _ in range(n + 1)]

    for i in range (1, n+1): # Find max value
        for j in range(1, w+1):
            if bkWeight[i -1] <= j:
                dp[i][j] = max(dp[i-1][j], bkVal[i - 1] + dp[i - 1][j - bkWeight[i - 1]])
            else:
                dp[i][j] = dp[i - 1][j]

    
    # Append function for the array
    def Append(A,i):
        B = [None] * (len(A)+1)

        for j in range(len(A)):
            B[j] = A[j]
        
        B[len(B)-1] = i

        return B
    
    def putInRightOrder(A):
        B = [None] * len(A)
        j = 0
        for i in range(len(A)-1, -1, -1):
            B[j] = A[i]
            j += 1
        
        return B

    # Go back and find which books were used.
    books_used = []
    remaining = w
    for i in range(n, 0, -1):
        if dp[i][remaining] != dp[i-1][remaining]:
            books_used = Append(books_used, i-1)
            remaining -= bkWeight[i-1]
    
    return dp[n][w], putInRightOrder(books_used)
    

# Test cases:
bkWeight = [1,1,3,4,6,12,33,45,50]
bkVal = [1,2,5,1,10,20,24,5,60]
print(maxBooks(100, bkWeight, bkVal))

(112, [0, 1, 2, 5, 6, 8])


# Question 5

Using dynamic programming, write a Python function: 

    def closestSubset(A,s)

that takes an array of non-negative integers `A` and a non-negative integer `s` and returns an array consisting 
of elements of `A` (i.e. a subset of `A`) which add up to `s`. If there is no subset that adds up to `s`, the function 
should instead return a subset which adds up to the value closest to `s`. 

For example: 

- if `A` is `[12, 79, 99, 91, 81, 47]` and `s` is `150`, it will return `[12, 91, 47]` as 12+91+47 is 150
- if `A` is `[15, 79, 99, 6, 69, 82, 32]` and `s` is `150` it will return `[69, 82]` as 69+82 is 151, and 
there is no subset of `A` whose sum is 150.

In more detail, your function should use an auxiliary function:

    def closestSubsetMem(A,s,lo,memo)
    
that returns an array consisting of elements of `A[lo:]` which add up to the closest value to `s`. To implement this function, you can use recursion as follows:

- If `s` is less or equal to 0 or `lo` is beyond the bounds of `A`, then the solution is simply `[]`
- Otherwise, we first consider the (recursive) case where element `A[lo]` is included in the selected elements (case withIt): 

    - if this case returns an array of elements that add up to `s` then the solution is that array
    - otherwise, we also consider the (recursive) case where element `A[lo]` is not included in the selected elements (case withoutIt); between the returned arrays of cases withIt and withoutIt, we select the one whose elements sum up closer to `s`.

Test the method with arrays generated by `randomIntArray(s,n)` from Lab 3. Try with:

    A = randomIntArray(20,1000)
    subset = closestSubset(A,5000)

In [11]:
import random

def closestSubset(A, s):
    memo = {}
    maxSum, sub = closestSubsetMem(A, s, 0, memo)
    return sub

def closestSubsetMem(A, s, lo, memo):
    if (s, lo) in memo:
        return memo[s, lo]

    if lo == len(A) or s == 0:
        return 0, []

    # Skip the current element to check without this element.
    sumWO, withoutItSub = closestSubsetMem(A, s, lo + 1, memo)

    
    sumW, withItSub = closestSubsetMem(A, s - A[lo], lo + 1, memo)
    sumW += A[lo]
    withItSub = withItSub + [A[lo]]

    
    # Find closest
    if abs(sumW - s) > abs(sumWO - s):
        result = (sumWO, withoutItSub)
    else:
        result = (sumW, withItSub)
    
    memo[s,lo] = result
    return result

# Test case
print(closestSubset([12, 79, 99, 91, 81, 47], 150))
print(closestSubset([15, 79, 99, 6, 69, 82, 32], 150))
def randomIntArray(size, valMax):
    A = [0] * size
    for i in range(size):
        A[i] = random.randint(1 , valMax)
    return A

A = randomIntArray(20,1000)
subset = closestSubset(A,5000)
print(subset)

[47, 91, 12]
[82, 69]
[527, 853, 298, 610, 21, 497, 702, 305, 636, 551]
